In [2]:
!git clone https://github.com/serengil/deepface.git
!cd deepface
!pip install -e 

Cloning into 'deepface'...

Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

-e option requires 1 argument


In [3]:
!pip install deepface opencv-python-headless mtcnn tf-fears

DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: Could not find a version that satisfies the requirement tf-fears (from versions: none)
ERROR: No matching distribution found for tf-fears


Defaulting to user installation because normal site-packages is not writeable


In [40]:
import cv2
import numpy as np
from deepface import DeepFace
from mtcnn import MTCNN
import numpy as np
from IPython.display import display, Image
import ipywidgets as widgets
import time

In [35]:
# Function to detect and crop faces using MTCNN
def detect_and_crop_face(image_path):
    detector = MTCNN()
    detections = detector.detect_faces(image)
    if detections:
        x, y, width, height = detections[0]['box']
        cropped_face = image[y:y+height, x:x+width]
        return cropped_face
    else:
        raise Exception("No face detected in the image")



In [27]:
# Function to preprocess images (grayscale, normalization, noise reduction, resize)
def preprocess_image(image, target_size=(224, 224)):
    # Convert to grayscale if necessary
    if len(image.shape) == 3 and image.shape[2] == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Normalize image
    image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)

    # Noise reduction using Gaussian Blur
    #image = cv2.GaussianBlur(image, (5, 5), 0)

    # Resize image
    image = cv2.resize(image, target_size)

    # Convert back to BGR for compatibility with models expecting 3 channels
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    return image


In [15]:
def verify_face(id_face, live_face):
    id_face_preprocessed = preprocess_image(id_face)
    live_face_preprocessed = preprocess_image(live_face)
    
    temp_id_image_path = 'temp_id_image.jpg'
    temp_live_image_path = 'temp_live_image.jpg'
    
    try:
        cv2.imwrite(temp_id_image_path, id_face_preprocessed)
        cv2.imwrite(temp_live_image_path, live_face_preprocessed)
        
        result = DeepFace.verify(img1_path=temp_id_image_path, img2_path=temp_live_image_path, model_name='ArcFace')
    finally:
        if os.path.exists(temp_id_image_path):
            os.remove(temp_id_image_path)
        if os.path.exists(temp_live_image_path):
            os.remove(temp_live_image_path)
    
    return result


In [38]:

# Load ID image
id_image_path = r'C:\Users\USER\Desktop\Screenshot 2023-12-22 002052.png'
image=cv2.imread(id_image_path)
id_face = detect_and_crop_face(id_image_path)
if id_face is None:
    raise ValueError("No face detected in ID image")

1/1 [==============================] - 0s 134ms/step


In [48]:
# Main script to capture and verify faces in real time
def capture_and_verify():
    cap = cv2.VideoCapture(0)  # Open the webcam
    if not cap.isOpened():
        print("Error: Could not open camera.")
        return

    while True:
        success, frame = cap.read()
        if not success:
            print("Error: Could not read frame from camera.")
            break
          # Display the frame
        cv2.imshow('Face Verification', frame)

        # Detect the face in the live frame
        live_face  = detect_and_crop_face(frame)
        if live_face is not None:
            # Verify the live face against the ID face
            result = verify_face(id_face, live_face)
            label = 'Verified' if result['verified'] else 'Not Verified'
            print(f"Verification Result: {label}")

            # Draw a rectangle around the detected face and display the result
            #x, y, width, height = box
            #cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
            #cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Display the frame
        cv2.imshow('Face Verification', frame)

        # Exit the loop when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        time.sleep(0.1)

    cap.release()
    cv2.destroyAllWindows()


In [49]:

# Run the capture and verify script
capture_and_verify()

1/1 [==============================] - 0s 111ms/step
Verification Result: Verified
1/1 [==============================] - 0s 110ms/step
Verification Result: Verified
1/1 [==============================] - 0s 133ms/step
Verification Result: Verified
1/1 [==============================] - 0s 119ms/step
Verification Result: Verified
1/1 [==============================] - 0s 136ms/step
Verification Result: Verified
1/1 [==============================] - 0s 125ms/step
Verification Result: Verified
1/1 [==============================] - 0s 130ms/step
Verification Result: Verified
1/1 [==============================] - 0s 154ms/step
Verification Result: Verified
1/1 [==============================] - 0s 130ms/step
Verification Result: Verified
1/1 [==============================] - 0s 150ms/step
Verification Result: Verified
1/1 [==============================] - 0s 111ms/step
Verification Result: Verified
1/1 [==============================] - 0s 144ms/step
Verification Result: Verified
1/1 